## Testing Autor results

First I am going to use his same process to get data

In [233]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import statistics
import datetime
import scipy.optimize as spopt
import datetime
import warnings


In [234]:
user=1
if user ==1:
    path = "/Users/lucadaquanno/Desktop/Documents/CIOS.Analyse/Return_forecasting/tests/Conditional_bootstrapping/Test_2/"

## Input generation

In [235]:
X=pd.read_excel(path+"test_Entropy_min_prob2.xlsx",header=None)
J=X.shape[0]

In [236]:
X.columns=['Stock1','Stock2']
start=datetime.datetime(2008,1,31)
end=datetime.datetime(2023,2,28)
t_index=pd.bdate_range(start,end)[:J]
X.index=t_index
X
p_0 = pd.Series(np.ones(J)/J,index=X.index) #equally weighted prior
p_0


2008-01-31    0.1
2008-02-01    0.1
2008-02-04    0.1
2008-02-05    0.1
2008-02-06    0.1
2008-02-07    0.1
2008-02-08    0.1
2008-02-11    0.1
2008-02-12    0.1
2008-02-13    0.1
Freq: B, dtype: float64

In [237]:


# set views and expectations
Q = np.array([1, -1])  # positive view on the first asset class, negative view on the second
Mu_Q = np.array([0.5])  # expectation on market returns
QX = np.array(np.dot(X, Q),ndmin=2)
# fix Mu_Q according to state-conditioning probabilities
# and then constrain Q*X to be equal to Mu_Q

# scatter
G = np.array([1, -1])  # views on the covariances
Sigma_G = 0.5 ** 2

# set constraints
Aeq = np.ones((1, J))  # constrain probabilities to sum to one...
beq = np.array([1])

# ...constrain the first moments...  
A = QX
b = Mu_Q

# ...constrain the second moments...
SecMom = np.array(np.dot(G, np.dot(Mu_Q, Mu_Q.T)).dot(G.T) + Sigma_G,ndmin=2)
GX = np.array(np.dot(X, G),ndmin=2)

for k in np.arange(G.shape[0]-1):
    for l in np.arange(k, G.shape[0]-1):
        A= np.vstack((A, (GX[k,:] * GX[l,:])))
        b = np.hstack((b, SecMom[k, l]))



In [238]:
p_0=pd.Series(p_0,index=X.index)
F=pd.DataFrame(A,columns=X.index,index=['Exp_value_const','Volatility_const'])
f=pd.Series(b,index=F.index)
H=pd.DataFrame(Aeq,columns=X.index,index=['ones'])
h=pd.Series(h,index=H.index)

In [239]:
K_eq=len(h)
K_ineq=len(f)
# initial guess
lmbda_vector_0=np.ones(K_eq+K_ineq)
lmbda_vector_0[K_eq:K_ineq+1]=-1 
lmbda_2=lmbda_vector_0[0:K_eq]       # Lagrange multipliers for equality constraints
lmbda_1=lmbda_vector_0[K_eq:K_ineq+1]# Lagrange multipliers for inequality constraints

In [240]:
def neg_Dual_func_eq_constr(Lmbda_vector,P_0,H_matrix,h):
   '''Lmbda_vector is a ndarray with (k_ineq + k_eq) number of elements
   P_0 is a series of prior probabilities with T number of elements
   H matrix must be a dataframe K_eq(number of equality constraints) rows and T columns (T number of scenarios)
   h is a series with equality constraints values
   lmbda vector is an array with initial values for Lagrange multipliers
   The function returns the objective function value to optimize '''
   K_eq=len(h)
   lmbda_2=Lmbda_vector[0:K_eq]
   Lmbda_vector[K_eq:]=0
   q=np.exp(np.log(P_0) - 1 - H_matrix.T.dot(lmbda_2))
   Dual_func= q.T.dot(np.log(q) - np.log(P_0)) + lmbda_2.T.dot(H_matrix.dot(q)-h)
   return - Dual_func


In [241]:
def neg_Dual_func_constr(Lmbda_vector,P_0,F_matrix,H_matrix,f,h):
   '''Lmbda_vector is a ndarray with (k_ineq + k_eq) number of elements
   P_0 is a series of prior probabilities with T number of elements
   F matrix must be a dataframe with K_ineq(number of inequality constraints) rows and T columns (T number of scenarios)
   H matrix must be a dataframe K_eq(number of equality constraints) rows and T columns (T number of scenarios)
   f is a series with intensity views for inequality constraints 
   h is a sereis with intensity views for equality constraints
   lmbda vector is an array with initial values for Lagrange multipliers
   The function returns the objective function value to optimize'''

   K_eq=len(h)
   K_ineq=len(f)
   lmbda_1=Lmbda_vector[K_eq:K_ineq+1]
   lmbda_2=Lmbda_vector[0:K_eq]
   q=np.exp(np.log(P_0) - 1 - F_matrix.T.dot(lmbda_1) - H_matrix.T.dot(lmbda_2))
   Dual_func=  q.T.dot(np.log(q) - np.log(P_0)) + lmbda_1.T.dot(F_matrix.dot(q)-f) + lmbda_2.T.dot(H_matrix.dot(q)-h)
   return - Dual_func

In [242]:
if (K_eq!=0) & (K_ineq!=0):
    obj_fun= neg_Dual_func_constr
else:
    obj_fun= neg_Dual_func_eq_constr

$\lambda_1(Fq - f)=0$

In [243]:
def lambda1_fun_eq(x,F_matrix,H_matrix,f,h):
     K_ineq=len(f)
     K_eq=len(h)
     lmbda_1=x[K_eq:K_ineq+1] # Lagrange multipliers for inequality constraints
     lmbda_2=x[0:K_eq]        # Lagrange multipliers for equality constraints
     q=np.exp(np.log(p_0) - 1 - F_matrix.T.dot(lmbda_1) - H_matrix.T.dot(lmbda_2))
     return lmbda_1*(F.dot(q)-f)



$Hq=h$

In [244]:
def lambda2_fun_eq(x,F_matrix,H_matrix,f,h,function=neg_Dual_func_constr):
    if function == neg_Dual_func_constr:
        K_ineq=len(f)
        K_eq=len(h)
        lmbda_1=x[K_eq:K_ineq+1] # Lagrange multipliers for inequality constraints
        lmbda_2=x[0:K_eq]        # Lagrange multipliers for equality constraints 
        q=np.exp(np.log(p_0) - 1 - F_matrix.T.dot(lmbda_1) - H_matrix.T.dot(lmbda_2))
    else:
         K_eq=len(h)
         lmbda_2=x[0:K_eq]
         q=np.exp(np.log(p_0) - 1 - H_matrix.T.dot(lmbda_2))
    return H.dot(q)- h

$ Fq - f \geq 0$

In [245]:
def ineq_cons(x,F_matrix,H_matrix,f,h):
    K_ineq=len(f)
    K_eq=len(h)
    lmbda_1=x[K_eq:K_ineq+1] # Lagrange multipliers for inequality constraints
    lmbda_2=x[0:K_eq]        # Lagrange multipliers for equality constraints 
    q=np.exp(np.log(p_0) - 1 - F_matrix.T.dot(lmbda_1) - H_matrix.T.dot(lmbda_2))
    return F.dot(q)-f

In [246]:
def lambda2_fun_eq(x,F_matrix,H_matrix,f,h,function=neg_Dual_func_constr):
    if function == neg_Dual_func_constr:
        K_ineq=len(f)
        K_eq=len(h)
        lmbda_1=x[K_eq:K_ineq+1] # Lagrange multipliers for inequality constraints
        lmbda_2=x[0:K_eq]        # Lagrange multipliers for equality constraints 
        q=np.exp(np.log(p_0) - 1 - F_matrix.T.dot(lmbda_1) - H_matrix.T.dot(lmbda_2))
    else:
         K_eq=len(h)
         lmbda_2=x[0:K_eq]
         q=np.exp(np.log(p_0) - 1 - H_matrix.T.dot(lmbda_2))
    return H.dot(q)- h

In [247]:
if (K_ineq!=0):
    cons =    ({'type': 'eq', 'fun': lambda1_fun_eq, 'args': (F,H,f,h)},
           {'type': 'eq', 'fun': lambda2_fun_eq, 'args': (F,H,f,h)},
           {'type': 'ineq', 'fun': ineq_cons,    'args': (F,H,f,h)})
    arguments=(p_0,F,H,f,h)
else: 
    cons = ({'type': 'eq', 'fun': lambda2_fun_eq, 'args': (F,H,f,h)})
    arguments=(p_0,H,h)

In [248]:
bnds= [(None,None),(None,0),(None,0)]

In [249]:
res=spopt.minimize(obj_fun,lmbda_vector_0,method='SLSQP',args=arguments,bounds=bnds,constraints=cons,options={'maxiter':100,'disp': True})
Lagrangian_mltps=res.x
lmbda_2=Lagrangian_mltps[0:K_eq]
lmbda_1=Lagrangian_mltps[K_eq:K_ineq+1]

Optimization terminated successfully    (Exit mode 0)
            Current function value: -0.04256564396053353
            Iterations: 12
            Function evaluations: 49
            Gradient evaluations: 12


In [250]:
Lagrangian_mltps=res.x
lmbda_2=Lagrangian_mltps[0:K_eq]
lmbda_1=Lagrangian_mltps[K_eq:K_ineq+1]
opt_lagran=res.x
post_prob=np.exp(np.log(p_0) - 1 - F.T.dot(lmbda_1) - H.T.dot(lmbda_2))
print(opt_lagran)
print(res.fun)
print(res)

[-9.56489145e-01 -1.72152998e-01 -4.12769775e-13]
-0.04256564396053353
     fun: -0.04256564396053353
     jac: array([-2.05449760e-06, -2.47796997e-05, -2.27933938e+00])
 message: 'Optimization terminated successfully'
    nfev: 49
     nit: 12
    njev: 12
  status: 0
 success: True
       x: array([-9.56489145e-01, -1.72152998e-01, -4.12769775e-13])


In [251]:
post_prob

2008-01-31    0.118959
2008-02-01    0.115314
2008-02-04    0.078174
2008-02-05    0.062067
2008-02-06    0.066833
2008-02-07    0.077056
2008-02-08    0.079492
2008-02-11    0.117259
2008-02-12    0.147689
2008-02-13    0.137156
Freq: B, dtype: float64